In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

precision = 'FP64'
bag = 10
stack = 2

prefix = f'{precision}_B{bag}S{stack}'
dataset = 'oneToTwo_dataset_V100'
save_path = f'/mnt/disk1/gjh/AutoML_models/agtModel_{prefix}_{dataset}'
train = TabularDataset(f'{precision}Train.csv')
test = TabularDataset(f'{precision}Test.csv')


In [ ]:
# import time
# agtPredictor = TabularPredictor.load(save_path)
# agtPredictor.persist_models(
#     models=['LightGBM_BAG_L1'], max_memory=0.5)
# tries = 200
# tmp = []
# s = time.perf_counter()
# for i in range(0, tries):
#     # s = time.perf_counter()
#     d = test.iloc[[i]]
#     t = agtPredictor.predict(d, model='LightGBM_BAG_L1')
#     # e = time.perf_counter()
#     # tmp.append((e-s)*1000)
# e = time.perf_counter()
# print(f"Run time per row: {(e-s)*1000/tries}ms")


In [ ]:
agtPredictor = TabularPredictor(label='label', path=save_path, problem_type='multiclass',
                                learner_kwargs={'ignored_columns': ['Matrix']}).fit(
    train, num_bag_folds=bag, num_stack_levels=stack, ag_args_fit={'num_gpus': 0})


In [ ]:
# t = agtPredictor.persist_models(
#     models=agtPredictor.get_model_names(), max_memory=0.5)


In [ ]:
agtPredictor = TabularPredictor.load(save_path)
a = agtPredictor.leaderboard(test, silent=True)
a10 = a.head(10)
a10 = a10[['model', 'score_test', 'score_val']]
a10plus = pd.DataFrame()
a10plus['model'] = a10['model']
best = a['model'][0]

In [ ]:
score_train = []
for model in a10plus['model']:
    st = agtPredictor.evaluate(train, model=model, silent=True)
    score_train.append(st[agtPredictor.eval_metric.name])
    print(model)


In [ ]:
a10plus['score_train'] = pd.Series(score_train)
af = a10.merge(a10plus)
af.to_csv(f'submit/{prefix}_{dataset}_score.csv', index=False)

In [ ]:
# c1 = agtPredictor.predict(train, model=best)
# c1 = pd.DataFrame(c1)
# c1['matrix'] = train['matrix']
# c1['group'] = train['group']
# c1.to_csv(f'submit2/{prefix}_pred_on_train.csv', index=False)

# agtPredictor = TabularPredictor.load(save_path)
c2 = agtPredictor.predict(test, model=best)
c2 = pd.DataFrame(c2)
c2['Matrix'] = test['Matrix']
# c2['group'] = test['group']
c2.to_csv(f'submit/{prefix}_pred_on_test_P100.csv', index=False)


In [ ]:
# # agtPredictor = TabularPredictor.load(save_path)
# a = agtPredictor.leaderboard(test, silent=True)
# b = a.head(10)[['model', 'pred_time_test', 'pred_time_test_marginal']]
# b.to_csv(f'submit2/{prefix}_pred_time_on_test.csv', index=False)
